In [1]:
#import packages

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import datetime

from library.sb_utils import save_file

In [2]:
#load in the raw data into dataframes
#both datasets are cumulative

us_vax_raw = pd.read_csv('data/us_state_vaccinations.csv', parse_dates=True)
us_cases_raw = pd.read_csv('data/us-states.csv', parse_dates=True)

In [3]:
#inspect the data

us_vax_raw.head()

,date,location,total_vaccinations,total_distributed,people_vaccinated,people_fully_vaccinated_per_hundred,total_vaccinations_per_hundred,people_fully_vaccinated,people_vaccinated_per_hundred,distributed_per_hundred,daily_vaccinations_raw,daily_vaccinations,daily_vaccinations_per_million,share_doses_used
0,2021-01-12,Alabama,78134.0,377025.0,70861.0,0.15,1.59,7270.0,1.45,7.69,NaN,NaN,NaN,0.207
1,2021-01-13,Alabama,84040.0,378975.0,74792.0,0.19,1.71,9245.0,1.53,7.73,5906.0,5906.0,1205.0,0.222
2,2021-01-14,Alabama,92300.0,435350.0,80480.0,NaN,1.88,NaN,1.64,8.88,8260.0,7083.0,1445.0,0.212
3,2021-01-15,Alabama,100567.0,444650.0,86956.0,0.28,2.05,13488.0,1.77,9.07,8267.0,7478.0,1525.0,0.226
4,2021-01-16,Alabama,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7557.0,7498.0,1529.0,NaN


total_vaccinations: total vaccinations administered
total_distributed: total vaccine doses distributed
people_vaccinated: total number of people who have received at least one dose
share_doses_used: percentage of distributed doses that have been administered (total_vaccinations / total_distributed)

In [4]:
us_vax_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11073 entries, 0 to 11072
Data columns (total 14 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   date                                 11073 non-null  object 
 1   location                             11073 non-null  object 
 2   total_vaccinations                   10738 non-null  float64
 3   total_distributed                    10563 non-null  float64
 4   people_vaccinated                    10551 non-null  float64
 5   people_fully_vaccinated_per_hundred  9944 non-null   float64
 6   total_vaccinations_per_hundred       10046 non-null  float64
 7   people_fully_vaccinated              10469 non-null  float64
 8   people_vaccinated_per_hundred        10021 non-null  float64
 9   distributed_per_hundred              10033 non-null  float64
 10  daily_vaccinations_raw               11008 non-null  float64
 11  daily_vaccinations          

In [5]:
us_cases_raw.head()

,date,state,fips,cases,deaths
0,2020-01-21,Washington,53,1,0
1,2020-01-22,Washington,53,1,0
2,2020-01-23,Washington,53,1,0
3,2020-01-24,Illinois,17,1,0
4,2020-01-24,Washington,53,1,0


In [6]:
#fips: regional code

In [7]:
us_cases_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26359 entries, 0 to 26358
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    26359 non-null  object
 1   state   26359 non-null  object
 2   fips    26359 non-null  int64 
 3   cases   26359 non-null  int64 
 4   deaths  26359 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 1.0+ MB


In [8]:
#exploring vaccination data based on state

us_vax_raw[us_vax_raw.location == 'New York State']

,date,location,total_vaccinations,total_distributed,people_vaccinated,people_fully_vaccinated_per_hundred,total_vaccinations_per_hundred,people_fully_vaccinated,people_vaccinated_per_hundred,distributed_per_hundred,daily_vaccinations_raw,daily_vaccinations,daily_vaccinations_per_million,share_doses_used
6800,2021-01-12,New York State,579532.0,1622100.0,543014.0,0.19,2.98,36422.0,2.79,8.34,NaN,NaN,NaN,0.357
6801,2021-01-13,New York State,632473.0,1796850.0,587865.0,0.23,3.25,44512.0,3.02,9.24,52941.00,52941.0,2721.0,0.352
6802,2021-01-14,New York State,688576.0,1872625.0,617596.0,NaN,3.54,NaN,3.17,9.63,56103.00,54522.0,2803.0,0.368
6803,2021-01-15,New York State,757466.0,1884325.0,671311.0,0.38,3.89,73523.0,3.45,9.69,68890.00,59311.0,3049.0,0.402
6804,2021-01-16,New York State,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60713.25,59662.0,3067.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6965,2021-06-26,New York State,21537500.0,23897255.0,11585130.0,53.10,110.71,10329090.0,59.55,122.84,73720.00,55120.0,2833.0,0.901
6966,2021-06-27,New York State,21684298.0,23897255.0,11614860.0,53.70,111.47,10446957.0,59.71,122.84,146798.00,67482.0,3469.0,0.907
6967,2021-06-28,New York State,21697307.0,23897255.0,11622076.0,53.75,111.53,10455550.0,59.74,122.84,13009.00,61244.0,3148.0,0.908
6968,2021-06-29,New York State,21748390.0,23944495.0,11645927.0,53.89,111.80,10484346.0,59.87,123.09,51083.00,63896.0,3285.0,0.908


In [9]:
#finding out how many unique locations are in each dataset

us_vax_raw.location.nunique()

65

In [10]:
us_cases_raw.state.nunique()

55

In [11]:
#finding the differences between the values in the two columns

np.setdiff1d(us_vax_raw.location.unique(), us_cases_raw.state.unique())

array(['American Samoa', 'Bureau of Prisons', 'Dept of Defense',
       'Federated States of Micronesia', 'Indian Health Svc',
       'Long Term Care', 'Marshall Islands', 'New York State',
       'Republic of Palau', 'United States', 'Veterans Health'],
      dtype=object)

In [12]:
np.setdiff1d(us_cases_raw.state.unique(), us_vax_raw.location.unique())

array(['New York'], dtype=object)

In [13]:
#replacing New York State with New York in the vaccination dataset for easier joining

us_vax_raw['location'].replace({'New York State': 'New York'}, inplace=True)

In [14]:
np.setdiff1d(us_vax_raw.location.unique(), us_cases_raw.state.unique())

array(['American Samoa', 'Bureau of Prisons', 'Dept of Defense',
       'Federated States of Micronesia', 'Indian Health Svc',
       'Long Term Care', 'Marshall Islands', 'Republic of Palau',
       'United States', 'Veterans Health'], dtype=object)

In [15]:
#drop rows with locations not found in the US cases dataset and reset the index

us_vax_raw.drop(us_vax_raw.loc[us_vax_raw['location'].isin(['American Samoa', 'Bureau of Prisons', 'Dept of Defense',
       'Federated States of Micronesia', 'Indian Health Svc',
       'Long Term Care', 'Marshall Islands', 'Republic of Palau',
       'United States', 'Veterans Health'])].index, inplace=True)
us_vax_raw.reset_index(drop=True, inplace=True)

In [16]:
us_vax_raw.columns

Index(['date', 'location', 'total_vaccinations', 'total_distributed',
       'people_vaccinated', 'people_fully_vaccinated_per_hundred',
       'total_vaccinations_per_hundred', 'people_fully_vaccinated',
       'people_vaccinated_per_hundred', 'distributed_per_hundred',
       'daily_vaccinations_raw', 'daily_vaccinations',
       'daily_vaccinations_per_million', 'share_doses_used'],
      dtype='object')

In [17]:
#dropping daily_vaccinations column because daily_vaccinations_raw already provides this info (unsure why the two are different values)

us_vax_raw.drop(columns='daily_vaccinations', inplace=True)

In [18]:
#fill in missing values for cumulative columns as previous value
#share_doses_used is a calculated field
#all filled values are grouped by the location

us_vax_raw[['total_vaccinations', 'total_distributed',
       'people_vaccinated', 'people_fully_vaccinated_per_hundred',
       'total_vaccinations_per_hundred', 'people_fully_vaccinated',
       'people_vaccinated_per_hundred', 'distributed_per_hundred']] = us_vax_raw.groupby('location')[['total_vaccinations', 'total_distributed',
       'people_vaccinated', 'people_fully_vaccinated_per_hundred',
       'total_vaccinations_per_hundred', 'people_fully_vaccinated',
       'people_vaccinated_per_hundred', 'distributed_per_hundred']].transform(lambda x: x.ffill())

us_vax_raw['share_doses_used'][us_vax_raw['share_doses_used'].isna()] = us_vax_raw['total_vaccinations'] / us_vax_raw['total_distributed']

<ipython-input-18-4a0fd8d2f205>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  us_vax_raw['share_doses_used'][us_vax_raw['share_doses_used'].isna()] = us_vax_raw['total_vaccinations'] / us_vax_raw['total_distributed']


In [19]:
us_vax_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9350 entries, 0 to 9349
Data columns (total 13 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   date                                 9350 non-null   object 
 1   location                             9350 non-null   object 
 2   total_vaccinations                   9350 non-null   float64
 3   total_distributed                    9350 non-null   float64
 4   people_vaccinated                    9338 non-null   float64
 5   people_fully_vaccinated_per_hundred  9317 non-null   float64
 6   total_vaccinations_per_hundred       9350 non-null   float64
 7   people_fully_vaccinated              9317 non-null   float64
 8   people_vaccinated_per_hundred        9338 non-null   float64
 9   distributed_per_hundred              9350 non-null   float64
 10  daily_vaccinations_raw               9295 non-null   float64
 11  daily_vaccinations_per_million

In [20]:
#investigate remaining rows that contain null values

pd.set_option('display.max_rows', None)
us_vax_raw[us_vax_raw.isnull().any(axis=1)]

,date,location,total_vaccinations,total_distributed,people_vaccinated,people_fully_vaccinated_per_hundred,total_vaccinations_per_hundred,people_fully_vaccinated,people_vaccinated_per_hundred,distributed_per_hundred,daily_vaccinations_raw,daily_vaccinations_per_million,share_doses_used
0,2021-01-12,Alabama,78134.0,377025.0,70861.0,0.15,1.59,7270.0,1.45,7.69,NaN,NaN,0.207000
170,2021-01-12,Alaska,35838.0,141600.0,22486.0,0.74,4.90,5400.0,3.07,19.36,NaN,NaN,0.253000
340,2021-01-12,Arizona,141355.0,563025.0,95141.0,0.11,1.94,8343.0,1.31,7.74,NaN,NaN,0.251000
510,2021-01-12,Arkansas,40879.0,274400.0,39357.0,0.00,1.35,8.0,1.30,9.09,NaN,NaN,0.149000
680,2021-01-12,California,816301.0,3286050.0,703540.0,0.25,2.07,100089.0,1.78,8.32,NaN,NaN,0.248000
850,2021-01-12,Colorado,224428.0,516750.0,187838.0,0.63,3.90,36425.0,3.26,8.97,NaN,NaN,0.434000
1020,2021-01-12,Connecticut,151395.0,250775.0,56350.0,0.12,4.25,4174.0,1.58,7.03,NaN,NaN,0.604000
1190,2021-01-12,Delaware,26410.0,88325.0,15911.0,0.29,2.71,2784.0,1.63,9.07,NaN,NaN,0.299000
1360,2021-01-12,District of Columbia,29812.0,62725.0,NaN,NaN,4.22,NaN,NaN,8.89,NaN,NaN,0.475000
1530,2021-01-12,Florida,633440.0,1676300.0,586741.0,0.19,2.95,41725.0,2.73,7.80,NaN,NaN,0.378000


In [21]:
#backfill remaining null values with the first available data from that row, grouped by location
#Guam has a lot of missing data for January, but it is so small that backfilling won't affect overall data that much

us_vax_raw[['people_vaccinated', 'people_fully_vaccinated_per_hundred',
       'people_fully_vaccinated','people_vaccinated_per_hundred',
       'daily_vaccinations_raw', 'daily_vaccinations_per_million']] = us_vax_raw.groupby('location')[['people_vaccinated', 'people_fully_vaccinated_per_hundred',
       'people_fully_vaccinated','people_vaccinated_per_hundred', 'daily_vaccinations_raw',
       'daily_vaccinations_per_million']].transform(lambda x: x.bfill())

In [22]:
#final inspection of vaccination data - no more null values

us_vax_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9350 entries, 0 to 9349
Data columns (total 13 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   date                                 9350 non-null   object 
 1   location                             9350 non-null   object 
 2   total_vaccinations                   9350 non-null   float64
 3   total_distributed                    9350 non-null   float64
 4   people_vaccinated                    9350 non-null   float64
 5   people_fully_vaccinated_per_hundred  9350 non-null   float64
 6   total_vaccinations_per_hundred       9350 non-null   float64
 7   people_fully_vaccinated              9350 non-null   float64
 8   people_vaccinated_per_hundred        9350 non-null   float64
 9   distributed_per_hundred              9350 non-null   float64
 10  daily_vaccinations_raw               9350 non-null   float64
 11  daily_vaccinations_per_million

In [23]:
#rename vaccination 'location' column to 'state' for easier merging

us_vax_raw.rename(columns={'location': 'state'}, inplace=True)

In [24]:
#combine the two datasets on date and state columns

combined_data = pd.merge(us_vax_raw, us_cases_raw, how='right', on=['date', 'state'])

In [25]:
#convert the 'date' column to datetime object

combined_data['date'] = pd.to_datetime(combined_data['date'])

In [26]:
#inspect the combined dataset - vaccination data will not show until 2021-01-12 but we can use the cases data to build a trend before vaccines were rolled out

combined_data.head()

,date,state,total_vaccinations,total_distributed,people_vaccinated,people_fully_vaccinated_per_hundred,total_vaccinations_per_hundred,people_fully_vaccinated,people_vaccinated_per_hundred,distributed_per_hundred,daily_vaccinations_raw,daily_vaccinations_per_million,share_doses_used,fips,cases,deaths
0,2020-01-21,Washington,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53,1,0
1,2020-01-22,Washington,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53,1,0
2,2020-01-23,Washington,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53,1,0
3,2020-01-24,Illinois,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17,1,0
4,2020-01-24,Washington,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53,1,0


In [27]:
combined_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26359 entries, 0 to 26358
Data columns (total 16 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   date                                 26359 non-null  datetime64[ns]
 1   state                                26359 non-null  object        
 2   total_vaccinations                   9020 non-null   float64       
 3   total_distributed                    9020 non-null   float64       
 4   people_vaccinated                    9020 non-null   float64       
 5   people_fully_vaccinated_per_hundred  9020 non-null   float64       
 6   total_vaccinations_per_hundred       9020 non-null   float64       
 7   people_fully_vaccinated              9020 non-null   float64       
 8   people_vaccinated_per_hundred        9020 non-null   float64       
 9   distributed_per_hundred              9020 non-null   float64       
 10  daily_vacc

In [28]:
#add new column with number of new cases per day

combined_data['new_cases'] = combined_data.groupby('state').diff(periods=1).cases

In [29]:
#for new cases < 0, make number of new cases equal to new cases from previous day

for i, row in combined_data.iterrows():
    if row['new_cases'] < 0:
        state = row['state']
        today = row['date']
        day_before = today - datetime.timedelta(days=1)
        
        # find yesterday's cases
        new_cases = combined_data[(combined_data['state'] == state) & 
                              (combined_data['date'] == day_before)]['new_cases'].values

        # set today's cases equal to yesterday's
        combined_data.loc[(combined_data['state'] == state) & 
                      (combined_data['date'] == today), 'new_cases'] = new_cases

In [30]:
#check if there are still new_cases less than 0

combined_data[combined_data['new_cases'] < 0]

,date,state,total_vaccinations,total_distributed,people_vaccinated,people_fully_vaccinated_per_hundred,total_vaccinations_per_hundred,people_fully_vaccinated,people_vaccinated_per_hundred,distributed_per_hundred,daily_vaccinations_raw,daily_vaccinations_per_million,share_doses_used,fips,cases,deaths,new_cases


In [31]:
#check which values in new_cases is null

combined_data[combined_data['new_cases'].isnull()]

,date,state,total_vaccinations,total_distributed,people_vaccinated,people_fully_vaccinated_per_hundred,total_vaccinations_per_hundred,people_fully_vaccinated,people_vaccinated_per_hundred,distributed_per_hundred,daily_vaccinations_raw,daily_vaccinations_per_million,share_doses_used,fips,cases,deaths,new_cases
0,2020-01-21,Washington,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53,1,0,NaN
3,2020-01-24,Illinois,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17,1,0,NaN
5,2020-01-25,California,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,1,0,NaN
8,2020-01-26,Arizona,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,1,0,NaN
35,2020-02-01,Massachusetts,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25,1,0,NaN
57,2020-02-05,Wisconsin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55,1,0,NaN
98,2020-02-12,Texas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48,1,0,NaN
133,2020-02-17,Nebraska,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31,10,0,NaN
199,2020-02-25,Utah,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49,1,0,NaN
225,2020-02-28,Oregon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41,1,0,NaN


In [32]:
#make the first number of new_cases equal to total cases for that day (currently null)

for i, row in combined_data.iterrows():
    if pd.isnull(row['new_cases']):
        combined_data.loc[i, 'new_cases'] = combined_data.loc[i, 'cases']

In [33]:
#check that there are no more null values in new_cases

combined_data[combined_data['new_cases'].isnull()]

,date,state,total_vaccinations,total_distributed,people_vaccinated,people_fully_vaccinated_per_hundred,total_vaccinations_per_hundred,people_fully_vaccinated,people_vaccinated_per_hundred,distributed_per_hundred,daily_vaccinations_raw,daily_vaccinations_per_million,share_doses_used,fips,cases,deaths,new_cases


In [34]:
#convert new_cases to integer value

combined_data['new_cases'] = combined_data['new_cases'].apply(np.int64)

In [35]:
combined_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26359 entries, 0 to 26358
Data columns (total 17 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   date                                 26359 non-null  datetime64[ns]
 1   state                                26359 non-null  object        
 2   total_vaccinations                   9020 non-null   float64       
 3   total_distributed                    9020 non-null   float64       
 4   people_vaccinated                    9020 non-null   float64       
 5   people_fully_vaccinated_per_hundred  9020 non-null   float64       
 6   total_vaccinations_per_hundred       9020 non-null   float64       
 7   people_fully_vaccinated              9020 non-null   float64       
 8   people_vaccinated_per_hundred        9020 non-null   float64       
 9   distributed_per_hundred              9020 non-null   float64       
 10  daily_vacc

In [36]:
#recalculate total cases equal to new cases + previous total cases

combined_data['total_cases'] = combined_data.groupby('state')['new_cases'].cumsum()

In [37]:
combined_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26359 entries, 0 to 26358
Data columns (total 18 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   date                                 26359 non-null  datetime64[ns]
 1   state                                26359 non-null  object        
 2   total_vaccinations                   9020 non-null   float64       
 3   total_distributed                    9020 non-null   float64       
 4   people_vaccinated                    9020 non-null   float64       
 5   people_fully_vaccinated_per_hundred  9020 non-null   float64       
 6   total_vaccinations_per_hundred       9020 non-null   float64       
 7   people_fully_vaccinated              9020 non-null   float64       
 8   people_vaccinated_per_hundred        9020 non-null   float64       
 9   distributed_per_hundred              9020 non-null   float64       
 10  daily_vacc

In [38]:
#drop the original cases column

combined_data.drop('cases', axis=1, inplace=True)

In [39]:
#create a dictionary with population of each state equal to people_vaccinated / people_vaccinated_per_hundred * 100
#used arbitrary date 06-01-2021

state_list = combined_data['state'].unique()
pop_dict = {}

for state in state_list:
    pop_dict[state] = (combined_data.loc[(combined_data['state'] == state) & 
                                        (combined_data['date'] == '06-01-2021'), 'people_vaccinated'].values //
                                        combined_data.loc[(combined_data['state'] == state) & 
                                        (combined_data['date'] == '06-01-2021'), 'people_vaccinated_per_hundred'].values * 100)[0].astype(np.int64)

In [40]:
#iterate through dataframe and assign the population of respective state to each row

for i, row in combined_data.iterrows():
    state = row['state']
    combined_data.loc[i, 'population'] = pop_dict[state]

In [41]:
#convert population column to int

combined_data['population'] = combined_data['population'].apply(np.int64)

In [42]:
combined_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26359 entries, 0 to 26358
Data columns (total 18 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   date                                 26359 non-null  datetime64[ns]
 1   state                                26359 non-null  object        
 2   total_vaccinations                   9020 non-null   float64       
 3   total_distributed                    9020 non-null   float64       
 4   people_vaccinated                    9020 non-null   float64       
 5   people_fully_vaccinated_per_hundred  9020 non-null   float64       
 6   total_vaccinations_per_hundred       9020 non-null   float64       
 7   people_fully_vaccinated              9020 non-null   float64       
 8   people_vaccinated_per_hundred        9020 non-null   float64       
 9   distributed_per_hundred              9020 non-null   float64       
 10  daily_vacc

In [43]:
datapath = './data'
save_file(combined_data, 'combined_data_cleaned.csv', datapath)

A file already exists with this name.

Do you want to overwrite? (Y/N)y
Writing file.  "./data\combined_data_cleaned.csv"
